In [1]:
import numpy as np
import uuid
import os
import torch
import pandas as pd
import requests
import customtkinter as ctk
from PIL import Image, ImageTk

num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

# Load the YOLOv5 model
default_model = torch.hub.load('ultralytics/yolov5', 'custom', path='bestdef.pt', force_reload=True) #depends on the exp number
orange_model = torch.hub.load('ultralytics/yolov5', 'custom', path='besto.pt', force_reload=True)

# Set confidence threshold
default_model.conf = 0.7
orange_model.conf = 0.5

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    # Move the model to CPU
    default_model = default_model.to('cpu')
    orange_model = orange_model.to('cpu')
    print("Models moved to CPU")
else:
    print("CUDA (GPU) is not available, model will run on CPU by default")

Number of GPUs available: 1


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\User/.cache\torch\hub\master.zip
YOLOv5  2024-3-22 Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)

Fusing layers... 
Model summary: 157 layers, 7050580 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\User/.cache\torch\hub\master.zip
YOLOv5  2024-3-22 Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)

Fusing layers... 
Model summary: 157 layers, 7047883 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


Models moved to CPU


In [2]:
import mediapipe as mp
import cv2 
import numpy as np
import uuid
import os
import torch
import serial.tools.list_ports
import pandas as pd
import requests
import time
import speech_recognition as sr
import urllib.request

In [3]:
ports=serial.tools.list_ports.comports()

portsList=[]
for onePort in ports:
    portsList.append(str(onePort))
    print(str(onePort))
    
#val1=input("First Port: COM")
val1=5
PortVar1='COM'+str(val1)

port1=serial.Serial(PortVar1, 9600, timeout=0.1)  
#port1.open()
#port1.close()

COM5 - Arduino Uno (COM5)


In [8]:
sending=True
voice_override=False
mp_drawing=mp.solutions.drawing_utils
mp_hands=mp.solutions.hands
mp_holistic=mp.solutions.holistic

def initial_scan(livespeed, cpd, spd, ppd, tpd, mode):
    command='turn left'
    (livespeed,cpd,spd,ppd,tpd,
    circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
    print(command)
    #send instruction
    bigmode=1
    if sending==True:
        pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)

# Gets the left and right hand (used by manual control)
def get_label(index, hand, results):

    output = None
    label= 'na'
    LC=tuple(np.multiply(
                    np.array((hand.landmark[mp_hands.HandLandmark.THUMB_IP].x, hand.landmark[mp_hands.HandLandmark.THUMB_IP].y)),
                    [640,480]).astype(int))
    RC=tuple(np.multiply(
                    np.array((hand.landmark[mp_hands.HandLandmark.PINKY_MCP].x, hand.landmark[mp_hands.HandLandmark.PINKY_MCP].y)),
                    [640,480]).astype(int))
    coords=(0,0)
    if index == 0:
        label = results.multi_handedness[0].classification[0].label
        score = results.multi_handedness[0].classification[0].score
        text = '{} {}'.format(label, round(score, 2))
        if label == 'Left':
            coords = LC
        if label == 'Right':
            coords = RC
    if index == 1:
        label = results.multi_handedness[1].classification[0].label
        score = results.multi_handedness[1].classification[0].score
        text = '{} {}'.format(label, round(score, 2))
        if label == 'Left':
            coords = LC
        if label == 'Right':
            coords = RC
    if label=='na':
        text='na'
    output = text, coords
    return output

def transcribe_microphone():
    # Initialize recognizer
    recognizer = sr.Recognizer()
    specific_class='None'
    # Use microphone as audio source
    with sr.Microphone() as source:
        print("Speak something...")
        recognizer.adjust_for_ambient_noise(source)  # Adjust for noise
        audio = recognizer.listen(source)  # Listen for audio input

    try:
        print("Transcribing...")
        text = recognizer.recognize_google(audio)  # Use Google Speech Recognition
        print("You said:", text)
        
        if 'override' in text.lower() or 'control' in text.lower() or 'manual' in text.lower():
            mode='manual'
            print("Command: manual")
            return mode # Return immediately after setting the mode

        elif 'default' in text.lower():
            mode='default'
            print('Command: default')
            return mode
        elif 'quit' in text.lower() or 'turn off' in text.lower():
            mode='quit'
            print('goodbye.')
            return mode

        #object
        trigger_words=['get', 'grab', 'fetch', 'pick up', 'bring']
        for trigger in trigger_words:
            if trigger in text.lower():
                #mode='YOLO'
                #check class
                classes = ['orange', 'fork', 'phone', 'can', 'mouse']
                specific_class = None
                for cls in classes:
                    if cls in text.lower():
                        specific_class = cls
                        break  # Select the first class instance and exit loop
                if specific_class:
                    mode=specific_class
                    print(f"Command: {trigger}, Class: {specific_class}")
                    return mode
                else:
                    print(f"No specific class mentioned after '{trigger}'.")
                    return None
                break  # Exit loop if any command is found
                
        print("No command detected.")
        
    except sr.UnknownValueError:
        print("Sorry, could not understand audio.")
    except sr.RequestError as e:
        print("Error:", e)
    return None
    
def extract_abcM(joint):
    a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y, hand.landmark[joint[0]].z]) # First coord
    b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y, hand.landmark[joint[1]].z]) # Second coord
    c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y, hand.landmark[joint[2]].z]) # Third coord
    M = np.array([(hand.landmark[joint[0]].x + hand.landmark[joint[2]].x)/2, 
                  (hand.landmark[joint[0]].y + hand.landmark[joint[2]].y)/2,
                  (hand.landmark[joint[0]].z + hand.landmark[joint[2]].z)/2]) # mid point coord
    J = np.array([M[0], M[1], a[2]]) # J = point on xy plane with 'a', paralell z-axis to M
    
    return a,b,c,M,J

def next_angle(angle_read_from_mediapipe, joint_angle_from_memory):
    stored_target= round(angle_read_from_mediapipe/ 10) * 10    # get nearest 10 degree
    if abs(joint_angle_from_memory-stored_target)>=6:   # check prev vs target
        
        if joint_angle_from_memory>stored_target:      #find direction to adjust
            output_angle=joint_angle_from_memory-2     #output it
        elif joint_angle_from_memory<stored_target:
            output_angle=joint_angle_from_memory+2     #output it
       
        #print(previousj1,j1,stored_target,outputj1) 
        
        joint_angle_from_memory=output_angle              #record the angle to memory
    else:                                           #angle near target
        output_angle=joint_angle_from_memory               #use angle from memory
        
    return output_angle, joint_angle_from_memory

def render_result(image,actual_text,text_position):
    cv2.putText(image, actual_text, tuple(np.multiply(text_position, [640, 480]).astype(int)),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

def pass_instruction_base(bigmode, Circle_Dir,Square_Dir,Plus_Dir,Triangle_Dir, Speed):
    Instruction = f"{bigmode} {Circle_Dir} {Square_Dir} {Plus_Dir} {Triangle_Dir} {Speed} "
    port1.write(Instruction.encode('utf-8'))
    
def pass_instruction_arm(bigmode, previousj1,  previousj2, previousj3, previousj4, pointless):
    Instruction = f"{bigmode} {previousj1} {previousj2} {previousj3} {previousj4} {pointless}"
    port1.write(Instruction.encode('utf-8'))
  
def fold_angle(a,b,c):
    px=abs(c[0]-a[0])
    py=abs(c[1]-a[1])
    pz=abs(c[2]-a[2])
    L2=np.linalg.norm(b - c)
    L3=np.linalg.norm(b - a)
    fold_angle=np.arccos((px**2+py**2+pz**2-L3**2-L2**2)/(2*L3*L2))
    fold_angle= fold_angle*180/np.pi
    
    return fold_angle

def curled_finger(a,b,c):
    px=abs(c[0]-a[0])
    py=abs(c[1]-a[1])
    pz=abs(c[2]-a[2])
    L2=np.linalg.norm(b - c)
    L3=np.linalg.norm(b - a)
    fold_angle=np.arccos((px**2+py**2+pz**2-L3**2-L2**2)/(2*L3*L2))
    fold_angle= fold_angle*180/np.pi
    
    if fold_angle>100:
        return True
    else:
        return False
    
def straight_finger(a,b,c):
    px=abs(c[0]-a[0])
    py=abs(c[1]-a[1])
    pz=abs(c[2]-a[2])
    L2=np.linalg.norm(b - c)
    L3=np.linalg.norm(b - a)
    fold_angle=np.arccos((px**2+py**2+pz**2-L3**2-L2**2)/(2*L3*L2))
    fold_angle= fold_angle*180/np.pi
    
    if fold_angle<30:
        return True
    else:
        return False
    
def fingers_straight():
    joint=[20,18,17]
    a,b,c,M,J = extract_abcM(joint)    #20,18,17
    #con3 pinky down
    con3=straight_finger(a,b,c)
    
    joint=[16,14,13]
    a,b,c,M,J = extract_abcM(joint) #16,14,13
    #con4 ring down
    con4=straight_finger(a,b,c)
   
    joint=[12,10,9]
    a,b,c,M,J = extract_abcM(joint)  #12,10,9
    #con5 middle down
    con5=straight_finger(a,b,c)
        
    joint=[8,6,5]
    a,b,c,M,J = extract_abcM(joint)   #8,6,5 
    #con6 index down   
    con6=straight_finger(a,b,c)
    
    if con3==True and con4==True and con5==True and con6==True:
        return True
    else:
        return False
    
def fingers_curled():
    joint=[20,18,17]
    a,b,c,M,J = extract_abcM(joint)    #20,18,17
    #con3 pinky down
    con3=curled_finger(a,b,c)
    
    joint=[16,14,13]
    a,b,c,M,J = extract_abcM(joint) #16,14,13
    #con4 ring down
    con4=curled_finger(a,b,c)
   
    joint=[12,10,9]
    a,b,c,M,J = extract_abcM(joint)  #12,10,9
    #con5 middle down
    con5=curled_finger(a,b,c)
        
    joint=[8,6,5]
    a,b,c,M,J = extract_abcM(joint)   #8,6,5 
    #con6 index down   
    con6=curled_finger(a,b,c)
    
    if con3==True and con4==True and con5==True and con6==True:
        return True
    else:
        return False
 
def wrist_vert_r(a,b,c,J,M):
    if a[0]< c[0] and  a[2]>c[2]:  #palm facing camera  
        radians = abs(np.arctan2((a[0] - J[0]), (J[2]-M[2])))  #(x5-xJ)/(zJ-zM)
        j1 = 90+(radians*180.0/np.pi)
    elif a[0]>c[0] and a[2]>c[2]:  #palm facing person 
        radians = (np.arctan2((a[0] - J[0]), (J[2]-M[2])))  #(x5-xJ)/(zJ-zM)
        j1 = 90-(radians*180.0/np.pi)
    elif a[0]< c[0] and  a[2]<c[2]:
        j1=180
    elif a[0]>c[0] and a[2]<c[2]:
        j1=0
    return j1

def wrist_vert_l(a,b,c,J,M):
    if a[0]< c[0] and  a[2]>c[2]:  #palm facing camera  
        radians = abs(np.arctan2((a[0] - J[0]), (J[2]-M[2])))  #(x5-xJ)/(zJ-zM)
        j1 = 90-(radians*180.0/np.pi)
    elif a[0]>c[0] and a[2]>c[2]:  #palm facing person 
        radians = (np.arctan2((a[0] - J[0]), (J[2]-M[2])))  #(x5-xJ)/(zJ-zM)
        j1 = 90+(radians*180.0/np.pi)
    elif a[0]< c[0] and  a[2]<c[2]:
        j1=0
    elif a[0]>c[0] and a[2]<c[2]:
        j1=180
    return j1

def j2_r(a,b,c,J,M,K):
    if M[2]>b[2] and a[0]>c[0] and M[1]>b[1]:   #back down
        j2=0
    elif M[2]<b[2] and a[0]<c[0] and M[1]>b[1]:   #front down
        j2=0
    elif M[2]>b[2] and a[0]<c[0] and M[1]>b[1]:  #back up
        j2=180
    elif M[2]<b[2] and a[0]>c[0] and M[1]>b[1]:  #front  up
        j2=180
    else:
        j2=fold_angle(M,b,K)
        
    return j2

def j2_l(a,b,c,J,M,K):
    if M[2]>b[2] and a[0]<c[0] and M[1]>b[1]:  #back down
        j2=0
    elif M[2]<b[2] and a[0]>c[0] and M[1]>b[1]:  #front down
        j2=0
    elif M[2]>b[2] and a[0]>c[0] and M[1]>b[1]: #back up
        j2=180
    elif M[2]<b[2] and a[0]<c[0] and M[1]>b[1]:  #front up
        j2=180 
    else:
        j2=fold_angle(M,b,K)
    
        
    return j2

def manual_r(image, results, previousj1, previousj2, previousj3):
    #joint 1

    joint=[5,0,17]     #5,0,17
    a,b,c,M,J= extract_abcM(joint)           #extract abcMJ
    text_position = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y])# position of text , Second coord
    j1=wrist_vert_r(a,b,c,J,M)           #calculates joint angle
    outputj1,previousj1 = next_angle(j1,previousj1)    #smoothing
    render_result(image,str(round(outputj1, 0)),text_position) 

    #joint 2
    joint=[5,0,17] 
    a,b,c,M,J = extract_abcM(joint)
    text_position = np.array([M[0],M[1]])# Second coord
    cj1=j1*np.pi/180
    K=np.array([b[0]+0.2*np.sin(np.pi-cj1), b[1], 0.2*np.cos(np.pi-cj1)+b[2]])
    j2=j2_r(a,b,c,J,M,K)
    text_positionk=np.array([K[0],K[1]])
    outputj2,previousj2 = next_angle(j2,previousj2) 
    render_result(image,str(round(outputj2, 0)),text_position)
    render_result(image,'K',text_positionk)
    
    
    #joint 3
    joint=[12,9,0]
    a,b,c,M,J = extract_abcM(joint)
    text_position = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y])
    j3=fold_angle(a,b,c)
    outputj3,previousj3 = next_angle(j3,previousj3) 
    render_result(image,str(round(outputj3, 0)),text_position) 
    
    
    #sending
    outputj1 = round(int(outputj1))
    outputj2 = round(int(outputj2))
    outputj3 = round(int(outputj3))
    #Combine the joint angles into a single message
   
    return image,outputj1,outputj2,outputj3

def manual_l(image, results, livespeed):
        
    command='none'
    #stop
    #palm has to face inside (con1)
    
    joint=[5,0,17]
    a,b,c,M,J = extract_abcM(joint)   #5,0,17
    text_position_j1= np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y])# Second coord
    j1=wrist_vert_l(a,b,c,J,M)
    render_result(image,str(round(j1, 0)),text_position_j1) 

    #con2 palm upright
    joint=[5,0,17]
    a,b,c,M,J = extract_abcM(joint)   #5,0,17
    text_position = np.array([M[0],M[1]])
    cj1=j1*np.pi/180
    K=np.array([b[0]-0.2*np.sin(np.pi-cj1), b[1], b[2]+0.2*np.cos(np.pi-cj1)])
    j2=j2_l(a,b,c,J,M,K)
    text_positionk=np.array([K[0],K[1]])
    render_result(image,str(round(j2, 0)),text_position)
    render_result(image,'K',text_positionk)
    
    #check fingers curled
    check_fingers_curled=fingers_curled()
    check_fingers_straight=fingers_straight()
    
    if 60<j1<120 and 60<j2<120 and check_fingers_curled==True:
        #render the result on the image
        command='stop'
        render_result(image,'stop',text_position_j1)
        
    joint=[12,9,0]
    a,b,c,M,J = extract_abcM(joint)  #12,9,0
    text_position = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y])
    j3=fold_angle(a,b,c)
    render_result(image,str(round(j3, 0)),text_position)
    
    if j1<20 and j2>160 and check_fingers_curled==True:
        command='turn left'
    if j1>160 and j2<50 and check_fingers_curled==True:
        command='turn right'
    
        
    if j1<30 and 70<j2<150 and j3<40:
        command='rev'
        render_result(image,'rev',text_position) 
    elif j1>160 and 75<j2<120 and 30<j3<90 and a[2]<c[2]:
        command='forward'
        render_result(image,'forward',text_position) 
    if command=='rev' or command=='stop' or command=='forward' or command=='turn left' or command=='turn right' or command=='left' or command=='right':
        hi='hi'
    else:
        command = 'nothing'
        
    return image,command

def come_follow(image,results, text, counter, hand, menu, grab):
    
    command='none'     #value for first reference
    j1=0
    j2=0
    j3=0

        
    if text[0]=='R':
        joint=[5,0,17]    #5,0,17
        a,b,c,M,J = extract_abcM(joint)
        text_position_j1 = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y])# position of text , Second coord
        j1=wrist_vert_r(a,b,c,J,M)
        render_result(image,str(round(j1, 0)),text_position_j1)
        center_coordinatex=int(round((a[0]+b[0]+c[0])/3*640,0))
        center_coordinatey=int(round((a[1]+b[1]+c[1])/3*480,0))
        
        
        joint=[5,0,17] 
        a,b,c,M,J = extract_abcM(joint)
        text_position = np.array([M[0],M[1]])# Second coord
        K = np.array([M[0], b[1], M[2]]) # K coord
        j2=j2_r(a,b,c,J,M,K)
        render_result(image,str(round(j2, 0)),text_position)
        
        #check fingers curled/straight
        check_fingers_curled=fingers_curled()
        check_fingers_straight=fingers_straight()
        
        if 60<j1<120 and 60<j2<120 and check_fingers_curled==True:
            #render the result on the image
            command='stop'
            render_result(image,'stop',text_position_j1)
            grab=False
        
        joint=[12,9,0]
        a,b,c,M,J = extract_abcM(joint)
        text_position = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y])
        j3=fold_angle(a,b,c)
        render_result(image,str(round(j3, 0)),text_position)
        
        
        if j1<30 and 70<j2<110 and check_fingers_straight==True:
            menu=True
        
        if menu==True:
            if j2<70 or j2>110 or j1>30:
                menu=False
                
        if menu==True:
            cv2.circle(image,(center_coordinatex,center_coordinatey), 20, (0,255,255), -1)
            if j1<30 and 70<j2<110 and check_fingers_curled==True:
                grab=True
                
        if grab==True:
            cv2.circle(image,(center_coordinatex,center_coordinatey), 20, (255,0,255), -1)
       # if grab==True:
            #check_fingers_straight=fingers_straight()
           # if check_fingers_straight==True:
              #  grab=False
                
                
    elif text[0]=='L':
        joint=[5,0,17]
        a,b,c,M,J = extract_abcM(joint)   #5,0,17
        text_position_j1 = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y])# Second coord
        j1=wrist_vert_l(a,b,c,J,M)
        render_result(image,str(round(j1, 0)),text_position_j1)
        center_coordinatex=int(round((a[0]+b[0]+c[0])/3*640,0))
        center_coordinatey=int(round((a[1]+b[1]+c[1])/3*480,0))
        
        joint=[5,0,17]
        a,b,c,M,J = extract_abcM(joint)
        text_position = np.array([M[0],M[1]])# Second coord
        K = np.array([M[0], b[1], M[2]]) # K coord
        j2=j2_l(a,b,c,J,M,K) 
        render_result(image,str(round(j2, 0)),text_position)
        
        #stop
        check_fingers_curled=fingers_curled()
        check_fingers_straight=fingers_straight()
        
        if 60<j1<120 and 60<j2<120 and check_fingers_curled==True:
            #render the result on the image
            command='stop'
            render_result(image,'stop',text_position_j1) 
            grab=False
         
        joint=[12,9,0]
        a,b,c,M,J = extract_abcM(joint)
        text_position = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y])
        j3=fold_angle(a,b,c)
        #render the result on the image
        render_result(image,str(round(j3, 0)),text_position)
        
        if j1<30 and 70<j2<110 and check_fingers_straight==True:
            menu=True
            
        if menu==True:
            if j2<70 or j2>110 or j1>30:
                menu=False
        if menu==True:
            cv2.circle(image,(center_coordinatex,center_coordinatey), 20, (0,255,255), -1)
            if j1<30 and 70<j2<110 and check_fingers_curled==True:
                grab=True
        
        if grab==True:
            cv2.circle(image,(center_coordinatex,center_coordinatey), 20, (255,0,255), -1)
            
        #if grab==True:
           # check_fingers_straight=fingers_straight()
            #if check_fingers_straight==True:
              #  grab=False
                
                
    #check command
    if j1>150 and 70<j2<110 and j3<28:     #hand in frame, come/follow
        if abs(a[0] - c[0])< abs(a[1] - c[1]):
            d1= abs(a[1] - c[1])
        elif  abs(a[0] - c[0])> abs(a[1] - c[1]):
            d1= abs(a[0] - c[0])
        #print(d1)
        #turn the direction
        if d1>0.5 :
            command='stop'
            render_result(image,'    too close, stop',text_position)
        elif b[0]>0.55:
            #turnright
            command='turn right'
            render_result(image,'    turn right,come',text_position)
        elif b[0]<0.45:
        #turn left
            command='turn left'
            render_result(image,'    turn left,come',text_position)
        elif 0.47<b[0]<0.53:
            #stop
            command='forward'                               #needs verification!!!!!!!!!!
            render_result(image,'    come',text_position)
        
    # option follow landmark
    
        
    if command=='follow':   #hand not in frame, holistic landmark (command registered)
        hi='hi'
        
    if command=='rev' or command=='stop' or command=='turn right' or command=='turn left' or command=='forward':
        hi='hi'
    else:
        command='nothing'
        
    return image, command, counter, menu, grab

def driving(command,livespeed,cpd,spd,ppd,tpd,mode):

    if command=='rev':
        circle_dir, square_dir,plus_dir,triangle_dir = 0,1,2,2
        livespeed=50
        cpd,spd,ppd,tpd=0,1,2,2
            
    elif command=='forward':
        circle_dir, square_dir,plus_dir,triangle_dir =  1,0,2,2
        livespeed=30
        cpd,spd,ppd,tpd=1,0,2,2
        
    elif command=='stop':
        circle_dir,square_dir,plus_dir, triangle_dir  = 2,2,2,2
        livespeed=0
        cpd,spd,ppd,tpd = 2,2,2,2
        
    elif command=='turn right':
        circle_dir,square_dir,plus_dir,triangle_dir= 1,1,1,1        
        livespeed=20
        cpd,spd,ppd,tpd=1,1,1,1
        
    elif command=='turn left':
        circle_dir,square_dir,plus_dir,triangle_dir  = 0,0,0,0        
        livespeed=20
        cpd,spd,ppd,tpd=0,0,0,0
        
    elif command=='left':
        circle_dir,square_dir,plus_dir,triangle_dir = 2,2,0,1        
        livespeed=25
        cpd,spd,ppd,tpd=2,2,0,1
    
    elif command=='right':
        circle_dir,square_dir,plus_dir,triangle_dir = 2,2,1,0        
        livespeed=25
        cpd,spd,ppd,tpd=2,2,1,0
        
    elif command=='nothing':
        if mode=='manual':
            if livespeed!=0:
                livespeed=livespeed-1
                
                circle_dir, circle_speed =  cpd, livespeed
                square_dir, square_speed =  spd, livespeed
                plus_dir, plus_speed =  ppd, 0
                triangle_dir, triangle_speed = tpd, 0
                
            else:
                circle_dir, square_dir,plus_dir, triangle_dir = cpd, spd, ppd, tpd

        elif mode=='default':
            #add check complete
             circle_dir, square_dir,plus_dir,triangle_dir = cpd,spd,ppd,tpd

        
    return (livespeed, cpd, spd, ppd, tpd,
            circle_dir,square_dir,plus_dir, triangle_dir)

def option_Menu():
    canvas=255* np.ones((640,480, 3), dtype=np.uint8)
    options= ['1. Manual mode', '2. Default mode', '3. YOLO', 'Q. Quit']
    for i, option in enumerate(options):
        cv2.putText(canvas, option, (50, 50+50*i), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    
    cv2.imshow('Menu', canvas)
    key=cv2.waitKey(0)
    
    if key== ord('1'):
        print('Selected manual mode')
        mode='manual'
        
        canvas.fill(255)
        submenu_options = ['Q. Quit']
        
        for i, option in enumerate(submenu_options):
            cv2.putText(canvas, option, (50,50+50*i), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    
        cv2.imshow('Menu', canvas)
        
        
    elif key== ord('2'):
        print('Selected default mode')
        mode='default'
        
        canvas.fill(255)
        submenu_options = ['Q. Quit']
        
        for i, option in enumerate(submenu_options):
            cv2.putText(canvas, option, (50,50+50*i), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    
        cv2.imshow('Menu', canvas)
    
    elif key== ord('3'):
        print('Selected YOLO')
        canvas.fill(255)
        yolo_options = ['1. can', '2. orange', '3. mouse', 'Q. Quit']
        
        for i, option in enumerate(yolo_options):
            cv2.putText(canvas, option, (50,50+50*i), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    
        cv2.imshow('Menu', canvas)
        key_yolo = cv2.waitKey(0)
        
        if key_yolo == ord('1'):
            print('Selected can')
            mode = 'can'
        elif key_yolo == ord('2'):
            print('Selected orange')
            mode = 'orange'
        elif key_yolo == ord('3'):
            print('Selected mouse')
            mode = 'mouse'
        elif key_yolo == ord('q'):
            print('Quitting')
            mode = 'quit'
        else:
            print('wrong input')
            mode = 'quit'
        
    elif key== ord('q'):
        print('Quitting')
        mode='quit'
        
    else:
        print('wrong input')
        mode=='quit'
        
    return mode


def person_tracking(width_p,height_p, sending, recenter, centroid_x, centroid_y, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4): 
    reached=False
    if recenter!=True:
    #x control (base)
        if centroid_x>340:
            command='turn right'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command)
            #send instruction
            bigmode=1
            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
        elif centroid_x<300:
            command='turn left'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command)
            #send instruction
            bigmode=1
            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
        elif 300 < centroid_x < 340 and (width_p>=300 and height_p>=160):
            command='stop'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command, 'approached person')
            #send instruction
            bigmode=1
            reached=True
            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
    
        elif 300 <= centroid_x <= 340 and (width_p<=300 and height_p<=160):
            command='forward'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command, 'found, approaching...')
            #send instruction
            bigmode=1

            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
                
    elif recenter==True:
        bigmode=0
        #y control (arm)
        if centroid_y>300: #lowerframe
            previousj2=previousj2-1   #move cam down
            print(previousj2)

            if sending==True:
                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed)     
        
        elif centroid_y<180: #upperframe
            if previousj2>=120:
                previousj3=previousj3-1  #move cam up

                if sending==True:
                    pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed)
            
            elif previousj2<120:
                previousj2=previousj2+1  # move cam up
                print(previousj2)

                if sending==True:
                    pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed)
                    

    return reached, previousj2, previousj3
    
def object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit): 
    reached=False
    if recenter!=True:
    #x control (base)
        if centroid_x>340:
            command='turn right'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command)
            #send instruction
            bigmode=1
            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
        elif centroid_x<300:
            command='turn left'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command)
            #send instruction
            bigmode=1
            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
        elif 300 < centroid_x < 340 and (width>=Ywid and height>=Yheit):
            command='stop'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command, 'approached object')
            #send instruction
            bigmode=1
            reached=True
            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
    
        elif 300 <= centroid_x <= 340 and (width<=Ywid and height<=Yheit):
            command='forward'
            (livespeed,cpd,spd,ppd,tpd,
            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
            print(command, 'found, approaching...')
            #send instruction
            bigmode=1

            if sending==True:
                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
                
    elif recenter==True:
        bigmode=0
        #y control (arm)
        if centroid_y>260: #lowerframe
            previousj2=previousj2-1   #move cam down
            print(previousj2)

            if sending==True:
                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed)     
        
        elif centroid_y<220: #upperframe
            if previousj2>=120:
                previousj3=previousj3-1  #move cam up

                if sending==True:
                    pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed)
            
            elif previousj2<120:
                previousj2=previousj2+1  # move cam up
                print(previousj2)

                if sending==True:
                    pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed)
                    

    return reached, previousj2, previousj3
    
condition='0'  
#mode
mode='nothing'
#run
run=True

previousj1,previousj2,previousj3,previousj4=90,125,150,90






while run==True:
    
    
    #setup
    command='nothing'
    livespeed = 0
    cpd,spd,ppd,tpd=2,2,2,2
    circle_dir,square_dir,triangle_dir,plus_dir=2,2,2,2
    menu,grab, havehandr,havehandl=False, False, False, False
    dl,dr=0,0
    condition='0' 
    
    
   
    if voice_override==True:
        mode=transcribe_microphone()
        print('mode: ', mode)
    else:
        mode=option_Menu()
        
    
    if mode=='default' or mode=='can' or mode=='mouse' or mode=='orange':
        #url='http://172.20.10.3:81/stream'#new code
        cap = cv2.VideoCapture(0)#new code
        #cap = cv2.VideoCapture(0)
        if not cap.isOpened():#new code
            print('failed to open IP camera')
            exit()

    if mode=='manual':
        url='http://172.20.10.3:81/stream'#new code
        cap = cv2.VideoCapture(url)
        #cap=cv2.VideoCapture(0)
        
        with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.3) as hands:

            while cap.isOpened():
                ret, frame = cap.read()         
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  #bgr2rgb
                image = cv2.flip(image, 1)              #flip
                image.flags.writeable = False           # Set flag
                results = hands.process(image)         # Detections
                image.flags.writeable = True             # Set flag to true
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   #rgb 2 bgr
                
                # Rendering results
                if results.multi_hand_landmarks:  # if any result
                    for num, hand in enumerate(results.multi_hand_landmarks):
                        mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=1),
                                                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1),)
                        
                
                        # Render left or right detection
                        if get_label(num, hand, results):
                            text, coord = get_label(num, hand, results)
                            cv2.putText(image, text, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                            if text[0]=='R':
                                image,previousj1,previousj2,previousj3 = manual_r(image, results,previousj1,previousj2,previousj3)
                                
                                joint=[5,0,17]
                                a,b,c,M,J = extract_abcM(joint)
                                d517=np.linalg.norm(a - c)
                                dM17=np.linalg.norm(M - b)
                                if d517< dM17:
                                    dr= dM17
                                elif  d517> dM17:
                                    dr= d517
                                center_coordinatexr=int(round((a[0]+b[0]+c[0])/3*640,0))
                                center_coordinateyr=int(round((a[1]+b[1]+c[1])/3*480,0))
                                havehandr=True
                                
                            if text[0]=='L':
                                image,command = manual_l(image, results, command)
                            
                                joint=[5,0,17]
                                a,b,c,M,J = extract_abcM(joint)
                                d517=np.linalg.norm(a - c)
                                dM17=np.linalg.norm(M - b)
                                if d517< dM17:
                                    dl= dM17
                                elif  d517> dM17:
                                    dl= d517
                                center_coordinatexl=int(round((a[0]+b[0]+c[0])/3*640,0))
                                center_coordinateyl=int(round((a[1]+b[1]+c[1])/3*480,0))
                                havehandl=True
                                
                            #print('dr:',dr,'dl:',dl)
                            
                else:
                    
                    command='nothing'

            
                #identify speed/direction     
                (livespeed,cpd,spd,ppd,tpd,
                circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)

                #send instruction
                
                
                if  havehandr==True and havehandl==True and dr>dl:
                    bigmode=0
                    cv2.circle(image,(center_coordinatexr,center_coordinateyr), 10, (255,0,255), -1)
                    
                    if dl!=0:
                        cv2.circle(image,(100,200), 10, (255,255,255), -1)
                        cv2.circle(image,(200,200), 10, (0,0,0), -1) 
                    if dl!=0 and 80<center_coordinatexl<120 and 180<center_coordinateyl<220:
                        previousj4=60
                    elif dl!=0 and 180<center_coordinatexl<220 and 180<center_coordinateyl<220:
                        previousj4=180 
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed) 
                        
                    havehandr,havehandl=False,False
                    
                elif  havehandl==True and havehandr==True and dl>dr:
                    bigmode=1
                    cv2.circle(image,(center_coordinatexl,center_coordinateyl), 10, (255,0,255), -1)
                    if sending==True:
                        pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir,triangle_dir, livespeed)
                    
                    havehandr,havehandl=False,False
                    
                elif havehandr==True and havehandl==False:
                    dl=0
                    bigmode=0
                    cv2.circle(image,(center_coordinatexr,center_coordinateyr), 10, (255,0,255), -1)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3,previousj4, livespeed)
                    
                    havehandr=False
                    
                elif havehandr==False and havehandl==True:
                    dr=0
                    bigmode=1
                    cv2.circle(image,(center_coordinatexl,center_coordinateyl), 10, (255,0,255), -1)
                    if sending==True:
                        pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir,triangle_dir, livespeed)
                    
                    havehandl=False
                
                cv2.namedWindow('manual control',cv2.WINDOW_NORMAL)
                cv2.resizeWindow('manual control',640,480)
                cv2.setWindowProperty('manual control',cv2.WND_PROP_TOPMOST, 1)
                cv2.imshow('manual control', image)

                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                # port1.close()
                    break
                    
                    
        cap.release()
        cv2.destroyWindow('manual control')


    elif mode=='default':   #come/folllow
        text='None'
        counter=0
        bigmode=0
        j2,j3=120,110
        while previousj2!=j2 and previousj3!=j3:
            if previousj2<j2:
                previousj2+=2
            elif previousj2>j2:
                previousj2-=2
            if previousj3<j3:
                previousj3+=2
            elif previousj3>j3:
                previousj3-=2
            print (previousj2,previousj3)
            if sending==True:
                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                delay_time = 0.05  # 0.05 s delay

                # Sleep for the specified delay time
                time.sleep(delay_time)
                
            
        with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.85) as holistic:
            #url='http://172.20.10.3:81/stream'#new code
           # cap = cv2.VideoCapture(url)#new code
            while cap.isOpened():
                recenter=False
                reached=False
                ret, frame = cap.read()
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  #bgr2rgb
                image.flags.writeable = False           # Set flag
                results=holistic.process(image)
                image.flags.writeable = True             # Set flag to true
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   #rgb 2 bgr
                image=cv2.resize(image,(640,480))

                if results.pose_landmarks:
                    pose=results.pose_landmarks
                    mp_drawing.draw_landmarks(image, pose, mp_holistic.POSE_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(247, 0, 255), thickness=2, circle_radius=1),
                                            mp_drawing.DrawingSpec(color=(255, 230, 0), thickness=2, circle_radius=1),)
                    

                if results.right_hand_landmarks:  # right hand
                    hand=results.right_hand_landmarks
                    mp_drawing.draw_landmarks(image, hand, mp_holistic.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=1),
                                            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1),)
                    joint=[12,9,0]
                    a,b,c,M,J = extract_abcM(joint)
                    if abs(a[0] - c[0])< abs(a[1] - c[1]):
                        dr= abs(a[1] - c[1])
                    elif  abs(a[0] - c[0])> abs(a[1] - c[1]):
                        dr= abs(a[0] - c[0])
                    
                    
                if results.left_hand_landmarks:   # left hand
                    hand=results.left_hand_landmarks
                    mp_drawing.draw_landmarks(image, hand, mp_holistic.HAND_CONNECTIONS, 
                                                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=1),
                                                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1),)
                    joint=[12,9,0]
                    a,b,c,M,J = extract_abcM(joint)
                    if abs(a[0] - c[0])< abs(a[1] - c[1]):
                        dl= abs(a[1] - c[1])
                    elif  abs(a[0] - c[0])> abs(a[1] - c[1]):
                        dl= abs(a[0] - c[0])

                if results.right_hand_landmarks and results.left_hand_landmarks:
                    if dl<dr:
                        text='Right'
                        hand=results.right_hand_landmarks
                    elif dr<dl:
                        text='Left'
                        hand=results.left_hand_landmarks
                    image, command, counter, menu, grab = come_follow(image, results, text, counter, hand, menu, grab)
                elif results.left_hand_landmarks:
                    text='Left'
                    hand=results.left_hand_landmarks
                    image, command, counter, menu, grab = come_follow(image, results, text, counter, hand, menu, grab)
                elif results.right_hand_landmarks:
                    text='Right'
                    hand=results.right_hand_landmarks
                    image, command, counter, menu, grab = come_follow(image, results, text, counter, hand, menu, grab)  
                    
                else:
                    command=='nothing'

                
                #new part
                if command=='stop':
                    stopcounter+=1
                elif command!='stop':
                    stopcounter=0
                print('stopcounter:', stopcounter)
                #end of new part

                
                if grab==True and results.pose_landmarks:
                    joint=[12,10,11]
                    M=np.array([(pose.landmark[joint[0]].x + pose.landmark[joint[2]].x)/2, (pose.landmark[joint[0]].y + pose.landmark[joint[2]].y)/2])
                    M_x=int(round(M[0]*640,0))
                    M_y=int(round(M[1]*480,0))
                    width_p=int(round(abs(pose.landmark[joint[0]].x-pose.landmark[joint[2]].x)*640,0))
                    height_p=int(round(abs(pose.landmark[joint[0]].y-pose.landmark[joint[1]].y)*480,0))
                    cv2.circle(image,(M_x,M_y), 20, (0,0,255), -1)
                    if M_y>300 or M_y<160: #if centroid y should be adjusted:
                        recenter=True
                    reached, previousj2, previousj3=person_tracking(width_p,height_p, sending, recenter, M_x,M_y, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4)
                    print(width_p,height_p)
                        
                    #if reached==True:
                        #print('reached')
                        #break  
                    
                    
                else:
                    (livespeed,cpd,spd,ppd,tpd,
                    circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)
                    #print(command)
                    #send instruction
                    bigmode=1
                    if sending==True:
                        pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)
                
                cv2.namedWindow('default mode',cv2.WINDOW_NORMAL)
                cv2.resizeWindow('default mode',640,480)
                cv2.setWindowProperty('default mode',cv2.WND_PROP_TOPMOST, 1)
                cv2.imshow('default mode', image)

                if cv2.waitKey(1) & 0xFF == ord('q') or stopcounter>=30:
                # port1.close()

                    break
                    
                    
        cap.release()
        cv2.destroyWindow('default mode')

        
        
    if mode=='mouse': 
        stopcounter=0
        specific_class='mouse'
        model=default_model
        Yj2 = 0
        Yj3 = 45
        Yj4_l = 160
        Yj4_s = 100
        Ywid = 170
        Yheit = 220
        min_distance_threshold = 10
        reached=False
        grabbed=False
        put_box=False
        dropped=False
        while cap.isOpened():
            
            ret, frame = cap.read()
            results= model(frame)
            boxes = results.xyxy[0].cpu().numpy()  # Convert to numpy array      
            class_to_priority = {} 
            recenter = False
            priority_mouse_present = False
            
            if reached==False:
                
                for box in boxes:
                    class_id = int(box[5])  # Extract class ID
                    class_name = model.names[class_id]  # Convert class ID to class name
                    confidence = box[4]

                    if class_name == specific_class:
                        centroid_x = (box[0] + box[2]) // 2  # Calculate centroid x-coordinate
                        centroid_y = (box[1] + box[3]) // 2  # Calculate centroid y-coordinate
                         # Check if the specific class is already assigned a priority
                        if specific_class not in class_to_priority:
                            # If not, assign the current object as the priority
                            class_to_priority[specific_class] = {'centroid': (centroid_x, centroid_y), 'confidence': confidence}
                        else:
                            # If yes, compare the confidence with the current object
                            if confidence > class_to_priority[specific_class]['confidence']:
                                # If current object has higher confidence, update priority
                                class_to_priority[specific_class] = {'centroid': (centroid_x, centroid_y), 'confidence': confidence}
                        if (centroid_x, centroid_y) == class_to_priority[specific_class]['centroid']:
                            priority_mouse_present = True

                for box in boxes:
                    class_id = int(box[5])  # Extract class ID
                    class_name = model.names[class_id]  # Convert class ID to class name
                    confidence = box[4]  # Extract confidence score
                    x1, y1, x2, y2 = box[:4].astype(int)
                    centroid_x = (x1 + x2) // 2
                    centroid_y = (y1 + y2) // 2
                    width = x2 - x1
                    height = y2 - y1

                    if class_name == specific_class:
                        # Check if the priority mouse is present in the current frame
                        if priority_mouse_present:
                            # If the priority mouse is present, mark it as red
                            if (centroid_x, centroid_y) == class_to_priority[specific_class]['centroid']:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                                print(f"Centroid position: ({centroid_x}, {centroid_y}, {width}, {height})")

                                if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                    recenter=True
                                reached, previousj2, previousj3 =object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)
                                if reached==True:
                                    print('reached')
                                    break  
                            else:
                                # Check if the distance between centroids is below the threshold
                                priority_centroid_x, priority_centroid_y = class_to_priority[specific_class]['centroid']
                                distance = np.sqrt((centroid_x - priority_centroid_x) ** 2 + (centroid_y - priority_centroid_y) ** 2)
                                if distance < min_distance_threshold:
                                    cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)  # Draw as red
                                    print(f"Centroid position: ({centroid_x}, {centroid_y})")

                                    if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                        recenter=True

                                    reached, previousj2, previousj3=object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)

                                    if reached==True:
                                        print('reached')
                                        break 
                                else:
                                    cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)  # Draw as green
                        else:
                            # If the priority mouse is not present, mark the first detected mouse as red
                            if confidence == class_to_priority[specific_class]['confidence']:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                                print(f"Centroid position: ({centroid_x}, {centroid_y})")

                                if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                    recenter=True

                                reached, previousj2, previousj3=object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)
                                if reached==True:
                                    print('reached')
                                    break
                            else:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)

                    else: 
                        stopcounter+=1
                        print('object lost')
                        if stopcounter>5:
                            command='stop' 
                            (livespeed,cpd,spd,ppd,tpd,
                            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)   
                            print(command, 'ped. Searching for object...')
                            #send instruction
                            bigmode=1

                            if sending==True:
                                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)

                            stopcounter=0
            elif reached==True:
                #grabbing
                
                
                bigmode=0
                if grabbed==False:
                    j2,j3,previousj4=Yj2,Yj3,Yj4_s
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                    print (previousj2,previousj3)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                    tolerance=3
                    if abs(previousj2-j2)<=tolerance  and abs(previousj3-j3)<=tolerance:
                        previousj4=Yj4_l
                        if sending==True:
                            for i in range(50):
                                
                                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                            
                            grabbed=True
                
                if grabbed==True and put_box==False:
                    #delay_time = 2  # 1-second delay

                            # Sleep for the specified delay time
                    #time.sleep(delay_time)
                    j2,j3=121,110
                    if previousj2<j2:
                        previousj2+=2
                    elif previousj2>j2:
                        previousj2-=2
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                    print (previousj2,previousj3)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                        
                    tolerance=3
                    if abs(previousj2-j2)<=tolerance  and abs(previousj3-j3)<=tolerance:
                        print('lifted')
                        put_box=True
                        
                if put_box==True and dropped==False:
                    
                    j1,j2,j3,j4=-10,130,130,Yj4_l
                    if previousj1<j1:
                        previousj1+=1
                    elif previousj1>j1:
                        previousj1-=1
                        
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                        
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                        
                    print (previousj1,previousj2,previousj3,previousj4)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                        
                    tolerance=4
                    if abs(previousj1-j1)<=tolerance and abs(previousj2-j2)<=tolerance  and abs(previousj3-j3)<=tolerance:
                        previousj4=Yj4_s
                        print('arrived')
                        if sending==True:
                            for i in range(50):
                                
                                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed)
                                print(previousj1, previousj2, previousj3, previousj4)
                            

                            dropped=True
                            
                if dropped==True:
                    j1,j2,j3,j4=90,130,130,Yj4_s
                    if previousj1<j1:
                        previousj1+=1
                    elif previousj1>j1:
                        previousj1-=1
                        
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                        
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                        
                    print (previousj1,previousj2,previousj3,previousj4)
                    if sending==True:

                            
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed)       
                            
            cv2.namedWindow('YOLO',cv2.WINDOW_NORMAL)
            cv2.imshow('YOLO', np.squeeze(results.render()))
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyWindow('YOLO')
    elif mode=='orange': 
        stopcounter=0
        specific_class='orange'
        model=orange_model
        Yj2 = 5
        Yj3 = 65
        Yj4_l = 160
        Yj4_s = 120
        Ywid = 210
        Yheit = 200 
        min_distance_threshold = 10
        reached=False
        grabbed=False
        put_box=False
        dropped=False
        while cap.isOpened():
            
            ret, frame = cap.read()
            results= model(frame)
            boxes = results.xyxy[0].cpu().numpy()  # Convert to numpy array      
            class_to_priority = {} 
            recenter = False
            priority_mouse_present = False
            
            if reached==False:
                
                for box in boxes:
                    class_id = int(box[5])  # Extract class ID
                    class_name = model.names[class_id]  # Convert class ID to class name
                    confidence = box[4]

                    if class_name == specific_class:
                        centroid_x = (box[0] + box[2]) // 2  # Calculate centroid x-coordinate
                        centroid_y = (box[1] + box[3]) // 2  # Calculate centroid y-coordinate
                         # Check if the specific class is already assigned a priority
                        if specific_class not in class_to_priority:
                            # If not, assign the current object as the priority
                            class_to_priority[specific_class] = {'centroid': (centroid_x, centroid_y), 'confidence': confidence}
                        else:
                            # If yes, compare the confidence with the current object
                            if confidence > class_to_priority[specific_class]['confidence']:
                                # If current object has higher confidence, update priority
                                class_to_priority[specific_class] = {'centroid': (centroid_x, centroid_y), 'confidence': confidence}
                        if (centroid_x, centroid_y) == class_to_priority[specific_class]['centroid']:
                            priority_mouse_present = True

                for box in boxes: 
                    class_id = int(box[5])  # Extract class ID
                    class_name = model.names[class_id]  # Convert class ID to class name
                    confidence = box[4]  # Extract confidence score
                    x1, y1, x2, y2 = box[:4].astype(int)
                    centroid_x = (x1 + x2) // 2
                    centroid_y = (y1 + y2) // 2
                    width = x2 - x1
                    height = y2 - y1

                    if class_name == specific_class:
                        # Check if the priority mouse is present in the current frame
                        if priority_mouse_present:
                            # If the priority mouse is present, mark it as red
                            if (centroid_x, centroid_y) == class_to_priority[specific_class]['centroid']:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                                print(f"Centroid position: ({centroid_x}, {centroid_y}, {width}, {height})")

                                if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                    recenter=True
                                reached, previousj2, previousj3 =object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)
                                if reached==True:
                                    print('reached')
                                    break  
                            else:
                                # Check if the distance between centroids is below the threshold
                                priority_centroid_x, priority_centroid_y = class_to_priority[specific_class]['centroid']
                                distance = np.sqrt((centroid_x - priority_centroid_x) ** 2 + (centroid_y - priority_centroid_y) ** 2)
                                if distance < min_distance_threshold:
                                    cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)  # Draw as red
                                    print(f"Centroid position: ({centroid_x}, {centroid_y})")

                                    if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                        recenter=True

                                    reached, previousj2, previousj3=object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)

                                    if reached==True:
                                        print('reached')
                                        break 
                                else:
                                    cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)  # Draw as green
                        else:
                            # If the priority mouse is not present, mark the first detected mouse as red
                            if confidence == class_to_priority[specific_class]['confidence']:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                                print(f"Centroid position: ({centroid_x}, {centroid_y})")

                                if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                    recenter=True

                                reached, previousj2, previousj3=object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)
                                if reached==True:
                                    print('reached')
                                    break
                            else:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)

                    else: 
                        stopcounter+=1
                        print('object lost')
                        if stopcounter>5:
                            command='stop' 
                            (livespeed,cpd,spd,ppd,tpd,
                            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)   
                            print(command, 'ped. Searching for object...')
                            #send instruction
                            bigmode=1

                            if sending==True:
                                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)

                            stopcounter=0
            elif reached==True:
                #grabbing
                
                
                bigmode=0
                if grabbed==False:
                    j2,j3,previousj4=Yj2,Yj3,Yj4_s
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                    print (previousj2,previousj3)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                    tolerance=3
                    if abs(previousj2-j2)<=tolerance  and abs(previousj3-j3)<=tolerance:
                        previousj4=Yj4_l
                        if sending==True:
                            for i in range(50):
                                
                                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 

                            
                        
                            grabbed=True
                
                if grabbed==True and put_box==False:
                    #delay_time = 2  # 1-second delay

                            # Sleep for the specified delay time
                    #time.sleep(delay_time)
                    j2,j3=121,110
                    if previousj2<j2:
                        previousj2+=2
                    elif previousj2>j2:
                        previousj2-=2
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                    print (previousj2,previousj3)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                        
                    tolerance=3
                    if abs(previousj2-j2)<=tolerance  and abs(previousj3-j3)<=tolerance:
                        print('lifted')
                        put_box=True
                        
                if put_box==True and dropped==False:
                    
                    j1,j2,j3,j4=-10,130,130,Yj4_l
                    if previousj1<j1:
                        previousj1+=1
                    elif previousj1>j1:
                        previousj1-=1
                        
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                        
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                        
                    print (previousj1,previousj2,previousj3,previousj4)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                        
                    tolerance=4
                    if abs(previousj1-j1)<=tolerance and abs(previousj2-j2)<=tolerance  and abs(previousj3-j3)<=tolerance:
                        previousj4=Yj4_s
                        print('arrived')
                        if sending==True:
                            for i in range(50):
                                
                                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed)
                                print(previousj1, previousj2, previousj3, previousj4)
                            

                            dropped=True
                            
                if dropped==True:
                    j1,j2,j3,j4=90,130,130,Yj4_s
                    if previousj1<j1:
                        previousj1+=1
                    elif previousj1>j1:
                        previousj1-=1
                        
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                        
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                        
                    print (previousj1,previousj2,previousj3,previousj4)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed)            
                        
            cv2.namedWindow('YOLO',cv2.WINDOW_NORMAL)
            cv2.imshow('YOLO', np.squeeze(results.render()))
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyWindow('YOLO')
    
    elif mode=='can': 
        stopcounter=0
        specific_class='can'
        model=default_model
        Yj2 = 35
        Yj3 = 85
        Yj4_l = 200
        Yj4_s = 100
        Ywid = 210
        Yheit = 270
        min_distance_threshold = 10
        reached=False
        grabbed=False
        put_box=False
        dropped=False

        look_for_object=True #new code  x 2
        while cap.isOpened():
            
            ret, frame = cap.read()
            results= model(frame)
            boxes = results.xyxy[0].cpu().numpy()  # Convert to numpy array      
            class_to_priority = {} 
            recenter = False
            priority_mouse_present = False
            
            if reached==False:
                if look_for_object==True:  #new code
                    initial_scan(livespeed, cpd, spd, ppd, tpd, mode)
                
                for box in boxes:
                    class_id = int(box[5])  # Extract class ID
                    class_name = model.names[class_id]  # Convert class ID to class name
                    confidence = box[4]

                    if class_name == specific_class:
                        centroid_x = (box[0] + box[2]) // 2  # Calculate centroid x-coordinate
                        centroid_y = (box[1] + box[3]) // 2  # Calculate centroid y-coordinate
                         # Check if the specific class is already assigned a priority
                        if specific_class not in class_to_priority:
                            # If not, assign the current object as the priority
                            class_to_priority[specific_class] = {'centroid': (centroid_x, centroid_y), 'confidence': confidence}
                        else:
                            # If yes, compare the confidence with the current object
                            if confidence > class_to_priority[specific_class]['confidence']:
                                # If current object has higher confidence, update priority
                                class_to_priority[specific_class] = {'centroid': (centroid_x, centroid_y), 'confidence': confidence}
                        if (centroid_x, centroid_y) == class_to_priority[specific_class]['centroid']:
                            priority_mouse_present = True

                for box in boxes:
                    class_id = int(box[5])  # Extract class ID
                    class_name = model.names[class_id]  # Convert class ID to class name
                    confidence = box[4]  # Extract confidence score
                    x1, y1, x2, y2 = box[:4].astype(int)
                    centroid_x = (x1 + x2) // 2
                    centroid_y = (y1 + y2) // 2
                    width = x2 - x1
                    height = y2 - y1

                    if class_name == specific_class:

                        look_for_object=False #new code  x 2
                        # Check if the priority mouse is present in the current frame
                        if priority_mouse_present:
                            # If the priority mouse is present, mark it as red
                            if (centroid_x, centroid_y) == class_to_priority[specific_class]['centroid']:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                                print(f"Centroid position: ({centroid_x}, {centroid_y}, {width}, {height})")

                                if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                    recenter=True
                                reached, previousj2, previousj3 =object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)
                                if reached==True:
                                    print('reached')
                                    break  
                            else:
                                # Check if the distance between centroids is below the threshold
                                priority_centroid_x, priority_centroid_y = class_to_priority[specific_class]['centroid']
                                distance = np.sqrt((centroid_x - priority_centroid_x) ** 2 + (centroid_y - priority_centroid_y) ** 2)
                                if distance < min_distance_threshold:
                                    cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)  # Draw as red
                                    print(f"Centroid position: ({centroid_x}, {centroid_y})")

                                    if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                        recenter=True

                                    reached, previousj2, previousj3=object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)

                                    if reached==True:
                                        print('reached')
                                        break 
                                else:
                                    cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)  # Draw as green
                        else:
                            # If the priority mouse is not present, mark the first detected mouse as red
                            if confidence == class_to_priority[specific_class]['confidence']:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)
                                print(f"Centroid position: ({centroid_x}, {centroid_y})")

                                if centroid_y>300 or centroid_y<160: #if centroid y should be adjusted:
                                    recenter=True

                                reached, previousj2, previousj3=object_tracking(sending, recenter, centroid_x, livespeed, cpd, spd, ppd, tpd, mode, previousj1, previousj2, previousj3,previousj4, Ywid, Yheit)
                                if reached==True:
                                    print('reached')
                                    break
                            else:
                                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)

                    else: 
                        stopcounter+=1
                        print('object lost')
                        if stopcounter>5:
                            command='stop' 
                            (livespeed,cpd,spd,ppd,tpd,
                            circle_dir,square_dir, plus_dir,triangle_dir) = driving(command, livespeed, cpd, spd, ppd, tpd, mode)   
                            print(command, 'ped. Searching for object...')
                            #send instruction
                            bigmode=1

                            look_for_object=True #new code  x 2

                            if sending==True:
                                pass_instruction_base(bigmode, circle_dir, square_dir, plus_dir, triangle_dir, livespeed)

                            stopcounter=0

            elif reached==True:
                #grabbing
                
                
                bigmode=0
                if grabbed==False and put_box==False:
                    j2,j3=Yj2,Yj3
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2

                    print (previousj2,previousj3,previousj4)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                    tolerance=3
                    if abs(previousj2-j2)<=tolerance and abs(previousj3-j3)<=tolerance:
                        j4=Yj4_l
                        if previousj4<j4:
                            previousj4+=5
                        elif previousj4>j4:
                            previousj4-=5
                        if sending==True:
                            pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed)
                            
                            if abs(previousj4-j4)<7:
                                grabbed=True
                
                if grabbed==True and put_box==False:
                    #delay_time = 2  # 1-second delay

                            # Sleep for the specified delay time
                    #time.sleep(delay_time)
                    j2,j3=121,110
                    if previousj2<j2:
                        previousj2+=2
                    elif previousj2>j2:
                        previousj2-=2
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                    print (previousj2,previousj3)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                        
                    tolerance=3
                    if abs(previousj2-j2)<=tolerance and abs(previousj3-j3)<=tolerance:
                        print('lifted')
                        put_box=True
                        
                if put_box==True and dropped==False:
                    
                    j1,j2,j3=-10,130,130
                    if previousj1<j1:
                        previousj1+=2
                    elif previousj1>j1:
                        previousj1-=2
                        
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                        
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                        

                        
                    print (previousj1,previousj2,previousj3,previousj4)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                        
                    tolerance=4
                    if abs(previousj1-j1)<=tolerance and abs(previousj2-j2)<=tolerance and abs(previousj3-j3)<=tolerance:
                        j4=Yj4_s
                        print('arrived')
                        
                        if previousj4<j4:
                            previousj4+=5
                        elif previousj4>j4:
                            previousj4-=5
                            
                        if sending==True:
                            pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed)
                            
                            if abs(previousj4-j4)<6:
                                dropped=True
                                
                            print(previousj1, previousj2, previousj3, previousj4)


                           
                            
                if dropped==True:
                    j1,j2,j3,j4=90,130,130,Yj4_s
                    if previousj1<j1:
                        previousj1+=2
                    elif previousj1>j1:
                        previousj1-=2
                        
                    if previousj2<j2:
                        previousj2+=1
                    elif previousj2>j2:
                        previousj2-=1
                        
                    if previousj3<j3:
                        previousj3+=2
                    elif previousj3>j3:
                        previousj3-=2
                        
                    print (previousj1,previousj2,previousj3,previousj4)
                    if sending==True:
                        pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 

                            
                            
            cv2.namedWindow('YOLO',cv2.WINDOW_NORMAL)
            cv2.imshow('YOLO', np.squeeze(results.render()))
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        cap.release()
        cv2.destroyWindow('YOLO')
    elif mode=='quit':
        bigmode=0
        j2,j3=120,150
        while (abs(previousj2-j2)>2) or (abs(previousj3-j3)>2):
            if previousj2<j2:
                previousj2+=2
            elif previousj2>j2:
                previousj2-=2
            if previousj3<j3:
                previousj3+=2
            elif previousj3>j3:
                previousj3-=2
            print (previousj2,previousj3)
            if sending==True:
                pass_instruction_arm(bigmode, previousj1, previousj2, previousj3, previousj4, livespeed) 
                delay_time = 0.05  # 0.05 s delay

                # Sleep for the specified delay time
                time.sleep(delay_time)
        #run=False
        break
cv2.destroyAllWindows()
#port1.close()

Selected manual mode
Quitting
95 84
97 86
99 88
101 90
103 92
105 94
107 96
109 98
111 100
113 102
115 104
117 106
119 108
121 110
119 112
121 114
119 116
121 118
119 120
121 122
119 124
121 126
119 128
121 130
119 132
121 134
119 136
121 138
119 140
121 142
119 144
121 146
119 148
